# Definition of the simulator class

In PHYSBO, the simulator class is used to define the objective function.
In this section, we will introduce how to define the simulator class based on an example of finding the minimum value of a one-dimensional function.

## Defining the search space

As in the other tutorials, the first step is to define the search space for the function.
In the following example, the search space ``X`` is defined as a grid chopped by ``window_num=10001`` divisions from ``x_min = -2.0`` to ``x_max = 2.0``.
Note that ``X`` must be in ``window_num`` x ``d`` ndarray format (``d`` is the number of dimensions, in this case two). In this case, ``d`` is the number of dimensions, in this case two, so we use reshape to transform it.

In [ ]:
#In
import numpy as np
import scipy
import physbo
import itertools

#In
#Create candidate
window_num=10001
x_max = 2.0
x_min = -2.0

X = np.linspace(x_min,x_max,window_num).reshape(window_num, 1)

## Defining the simulator class

Here, we define the simulator class to set as the objective function.

In this case, the problem is to find the minimum $x$ such that $f(x) = 3 x^4 + 4 x ^3 + 1.0$ (the answer is $x=-1.0$).

In the simulator class, we define the ``__call__`` function (or ``__init__`` if there are initial variables, etc.).
(If there are initial variables, define ``__init__``.) The action indicates the index number of the grid to be retrieved from the search space, and is generally in the form of an ndarray so that multiple candidates can be calculated at once.
In this case, we choose one candidate point from ``X`` as ``action_idx=action[0]`` to calculate only one candidate at a time.
Since PHYSBO is designed to find the one with the largest objective function value, it returns the value of f(x) at the candidate point multiplied by -1.

In [ ]:
# Declare the class for calling the simulator.
class simulator:

    def __call__(self, action ):
        action_idx = action[0]
        x = X[action_idx][0]
        fx = 3.0*x**4 + 4.0*x**3 + 1.0
        fx_list.append(fx)
        x_list.append(X[action_idx][0])

        print ("*********************")
        print ("Present optimum interactions")

        print ("x_opt=", x_list[np.argmin(np.array(fx_list))])

        return -fx, x

## Random Search

Before performing Bayesian optimization, prepare training data in advance by randomly selecting candidate points from the search space and evaluating the objective function value.

In [ ]:
fx_list=[]
x_list = []
#In
# Design of policy
# Declaring the policy by
policy = physbo.search.discrete.policy(test_X=X)
# test_X is the set of candidates which is represented by numpy.array.
# Each row vector represents the feature vector of the corresponding candidate

# set the seed parameter
policy.set_seed( 1 )


# If you want to perform the initial random search before starting the Bayesian optimization,
# the random sampling is performed by

res = policy.random_search(max_num_probes=50, simulator=simulator())
# Input:
# max_num_probes: number of random search
# simulator = simulator
# output: combo.search.discreate.results (class)

## Performing Bayesian optimization

Performs Bayesian optimization on the defined simulator.

In [ ]:
# single query Bayesian search
# The single query version of COMBO is performed by
res = policy.bayes_search(max_num_probes= 150, simulator=simulator(), score='TS',
                                                  interval=20, num_rand_basis=5000)

# Input
# max_num_probes: number of searching by Bayesian optimization
# simulator: the class of simulator which is defined above
# score: the type of aquision funciton. TS, EI and PI are available
# interval: the timing for learning the hyper parameter.
#               In this case, the hyper parameter is learned at each 20 steps
#               If you set the negative value to interval, the hyper parameter learning is not performed
#               If you set zero to interval, the hyper parameter learning is performed only at the first step
# num_rand_basis: the number of basis function. If you choose 0,  ordinary Gaussian process runs

## Plotting results

The results with the best score can be retrieved using ``export_all_sequence_best_fx()``.
If you want to see the entire history, you can use ``chosen_actions`` to display it. Here is a sample script to display the results.

In [ ]:
#In
best_fx, best_action = res.export_all_sequence_best_fx()

import matplotlib.pyplot as plt
# The result of searching is summarized in the class combo.search.discrete.results.history()
# res.fx: observed negative energy at each step
# res.chosen_actions: history of choosed actions
# fbest, best_action= res.export_all_sequence_best_fx(): current best fx and current best action
#                                                                                                   that has been observed until each step
# res.total_num_search: total number of search
plt.plot(res.fx[0:res.total_num_search])

In [ ]:
plt.plot(best_fx)

In [ ]:
plt.plot(best_action)

Finally, the candidate with the best score can be displayed as follows. You can see that we have arrived at the correct solution $x=-1$.

In [ ]:
print(X[int(best_action[-1])])